In [17]:
import os 
import imp
import warnings
import numpy as np
import pandas as pd
import importlib as imp
import itertools
from tqdm import tqdm, tqdm_notebook
from utils import tools
warnings.simplefilter('ignore')
pd.options.display.max_columns = 100

PATH_TO_BENCHMARKS = './/data//'

**Задача**

*Ваша четвертая лабораторная работа будет заключаться в реализации одного из двух алгоритмов: <br>
метод имитации обжига или генетический алгоритм со встроенной эвристикой в виде локального поиска.  <br>
Алгоритмы реализуются для задачи Cell formation problem.* <br>

Реализовать можно на любом языке. <br>
Также нужно будет сравнить время работы алгоритма, а так же показать лучшее решение алгоритма, которое удалось найти вам. <br>

**Данные:**

Данные для работы прикрепил архивом к письму. **Структура такая:** <br>

m p (number of machines and parts) <br>
Next m rows: <br>
m(row number) list of parts processed by machine m separated by space <br>
e.g: <br>
1 9 17 19 31 33
means machine 1 processes parts 9 17 19 31 33 <br>

**Выходной формат для ответов:** <br>
Output file: instancename.sol (e.g. 20x20.sol) <br>
Output file format: <br>
m1_clusterId m2_clusterId ... - machines to clusters mapping <br>
p1_clusterId p2_clusterId ... - parts to clusters mapping <br>

In [2]:
txtfiles = [f for f in os.listdir(PATH_TO_BENCHMARKS) if os.path.isfile(os.path.join(PATH_TO_BENCHMARKS, f))]
txtfiles

['20x20.txt', '24x40.txt', '30x50.txt', '30x90.txt', '37x53.txt']

In [3]:
tools = imp.reload(tools)
machine_part_matrix = tools.get_data(PATH_TO_BENCHMARKS + txtfiles[0])
machine_part_matrix_sample = np.array([[1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])

In [4]:
pd.DataFrame(machine_part_matrix_sample)

,0,1,2,3,4
0,1,0,0,1,0
1,0,1,1,0,1
2,1,0,0,0,0
3,0,1,1,0,0
4,0,0,0,1,0


Построение $similarity\ matrix$ по формуле $$S_{ij} = \frac{a_{ij}}{a_{ij} + b_{ij} + c_{ij}}$$
$a_{ij}$ - кол-во машин, которые обрабатывают детали $i, j$
$b_{ij}$ - кол-во машин, которые обрабатывают деталь $i$, но не $j$
$c_{ij}$ - кол-во машин, которые обрабатывают деталь $j$, но не $i$

In [5]:
similarity_matrix, np_similarity = tools.get_similarity_pairs(machine_part_matrix_sample)
cells_p = tools.get_assignment_matrix(np_similarity)

In [6]:
pd.DataFrame(similarity_matrix)

,0,1,2,3,4
0,-1.0,0.0,0.0,0.333333,0.0
1,-1.0,-1.0,1.0,0.000000,0.5
2,-1.0,-1.0,-1.0,0.000000,0.5
3,-1.0,-1.0,-1.0,-1.000000,0.0
4,-1.0,-1.0,-1.0,-1.000000,-1.0


Матрицу удобнее представить в виде списка: $[[(i, j), S_{i, j}]]$

In [7]:
np_similarity

[[(1, 2), 1.0], [(1, 4), 0.5], [(2, 4), 0.5], [(0, 3), 0.3333333333333333]]

Итерируясь в порядке убывания по списку составленному по $similarity\ matrix$ генерируем первоначальное решение (разбиение на $cells$)

In [8]:
cells_p

{0: [1, 2, 4], 1: [0, 3]}

$Cell 1:$ [1, 2, 4]
$Cell 2:$ [0, 3]

In [9]:
pd.DataFrame(machine_part_matrix_sample)[[1, 2, 4, 0, 3]]

,1,2,4,0,3
0,0,0,0,1,1
1,1,1,1,0,0
2,0,0,0,1,0
3,1,1,0,0,0
4,0,0,0,0,1


In [112]:
tools = imp.reload(tools)
machine_part_matrix = tools.get_data(PATH_TO_BENCHMARKS + txtfiles[4])
cells_p, cells_m = tools.get_solution(machine_part_matrix)
cells_p, cells_m

({0: [12, 18, 6, 9, 43, 44, 21, 11],
  1: [13, 15, 5, 17, 1],
  2: [19, 22, 3, 16],
  3: [26, 31, 25, 30, 29, 34, 52],
  4: [27, 35, 37, 32, 36],
  5: [10, 14, 20, 41, 40],
  6: [28, 38, 33, 45, 48, 49, 39],
  7: [7, 8, 4, 46, 47, 2, 51, 50],
  8: [0, 42, 23, 24]},
 {0: [7, 10, 13, 14, 16, 17, 18, 19, 20, 22, 25, 30, 32, 34],
  1: [2, 5, 6, 12, 15, 24, 28],
  2: [1, 4, 33],
  3: [0, 8, 21, 23, 26, 27, 29, 31],
  4: [36],
  5: [11],
  6: [3, 9],
  7: [35]})

In [113]:
tools.print_solution_df(cells_p, cells_m, machine_part_matrix)

,12,18,6,9,43,44,21,11,13,15,5,17,1,19,22,3,16,26,31,25,30,29,34,52,27,35,37,32,36,10,14,20,41,40,28,38,33,45,48,49,39,7,8,4,46,47,2,51,50,0,42,23,24
7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1
10,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
13,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
14,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,1,1
16,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
17,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,0,1,1,1,1,1
18,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,0,1,1,1,1,1
19,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,1,1,1
20,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
22,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,1,1,0,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1
